# Notebook \# 1: Extracción, transformación y carga de los datos

### Librerias útiles

Una vez que se cuenta con el conjunto de datos etiquetado, éste debe ser dividido en tres subconjuntos que llamaremos "entrenamiento", "validación" y "prueba", divididos en 70%, 10% y 20% de los datos. Esta división es común dentro de la ciencia de datos y está basada en el *principio de Pareto*.

Además de la división es necesario transformar el texto de cada tweet, pues recordemos cada texto puede contener emoticonos 🇧🇷🇨🇷🇳🇮🇲🇽, hashtags *#VivaLaMigración*, menciones a usuarios [@el_BID](https://twitter.com/el_bid?lang=es) y links a páginas externas. Algunos de estos elementos pueden contener información útil para la detección de sentimiento y/o xenofobia (como los hashtags), pero otros pueden no ser nada informativos (como los nombres de usuario).

##### Scikit learn
Se trata de una librería de código abierto enfocada en proveer herramientas de aprendizaje de máquinas tales como modelos estadísticos y matemáticos, así como métricas de evaluación comunes en algoritmos de aprendizaje de máquinas.
<figure>
    <img src="./assets/images/scikit.png"
         alt="scikit-learn logo"
         style="max-width: 40%; height: auto">
    <figcaption>scikit-learn logo</figcaption>
</figure>

Esta librería nos permitirá dividir el conjunto de datos en los subconjuntos mencionados.

In [1]:
#Imports

#data reading
import pandas as pd

#data processing
from pysentimiento.preprocessing import preprocess_tweet
import re

#scikit-learn creacion de conjuntos de entrenamiento, prueba y validación
from sklearn.model_selection import train_test_split

### Cargando los datos etiquetados
Se cargan los datos etiquetados en un DataFrame de pandas, el cual es una estructura de datos de alto rendimiento y fácil de usar, que nos permite manipular los datos de forma eficiente. Para esta ocasión solo se usan las columnas "id", "text" y "label" por motivos de simplicidad y confidencialidad. Los datos extraídos de Twitter se pueden recuperar a través del id del tweet y los datos guardadps en mongo DB.

In [2]:
data = pd.read_csv('./assets/data/xeno_data_workshop.csv')
data.sample(n=5)

,id,text,date,label
5401,1386572116670271488,@OviedoJoelys Desde Perú aquí una venezolana e...,2021-09-01,0
3922,1206336800282816512,"@yanferluis @virginiatreyes Amiga, no desmayes...",2021-09-04,0
6465,1114601492206497792,Eso es nada. Sólo en Venezuela hay más de 5.00...,2021-11-19,0
8149,1264945348499379968,@PeCazafantasmas @DanielaBrandon La verdad muy...,2021-06-29,0
9656,1294699845660413952,Los migrantes pobres del mundo si están sufri...,2021-10-19,0


Mostremos el texto de un tweet etiquetado como xenofóbico y otro como no xenofóbico.

In [3]:
print('Tweet marcado como xenofóbo:\n {}\nTweet marcado como NO xenofóbo:\n {}'.format(data[data.label==1].sample(n=1).text.values[0], data[data.label==0].sample(n=1).text.values[0]))

Tweet marcado como xenofóbo:
 @ahope71 @lopezobrador_ Es simple, parar los migrantes antes de llegar a México o siquiera mostrar la intención de hacerlo.
Tweet marcado como NO xenofóbo:
 Después de un 2018 de inmigrantes venezolanos tan fuerte viene una etapa más triste y de reencuentro.


### Limpieza de textos

Como ya se mencionó, cada texto puede contener emoticonos, hashtags, menciones a usuarios y links a páginas externas. Algunos de estos elementos pueden contener información útil para la detección de sentimiento y/o xenofobia (como los hashtags), pero otros pueden ser irrelevantes (como los nombres de usuario). Por lo tanto, es necesario limpiar el texto de cada tweet para que solo contenga palabras y signos de puntuación.

Para clarificar el proceso de procesamiento de texto, se muestra un texto de ejemplo, generado sintéticamente, el cual contiene emojis, hastags, expresiones de risa, links a páginas web. Este ejemplo fue generado a modo de capturar la mayor cantidad de elementos que se pueden encontrar en un tweet.

In [4]:
example_text = 'Que opinas de la reformas Migratorias?👀👀 mi buen amigo @Pepe?\nYo pienso que DEbieron facilitar la entrada de los inmigrantes 💁🇧🇷🇨🇷🇳🇮🇲🇽💪#YoTeApoyo eeeexxxxxx jajajajaja.'
bid_url = ' Página oficial del BID:          https://www.iadb.org/es '
example_text = example_text + bid_url
print(example_text)

Que opinas de la reformas Migratorias?👀👀 mi buen amigo @Pepe?
Yo pienso que DEbieron facilitar la entrada de los inmigrantes 💁🇧🇷🇨🇷🇳🇮🇲🇽💪#YoTeApoyo eeeexxxxxx jajajajaja. Página oficial del BID:          https://www.iadb.org/es 


El procesado de cada texto se deja a cargo del método `preprocess_tweet` el cual se encarga de transformar los emojis, hashtags, menciones a usuarios y links a páginas externas, normalizar las rizas y acortar palabras repetidas. Este método fue creado por el equipo de [Pysentimiento](https://github.com/pysentimiento/pysentimiento)

Este método permite al usuario definir la transformación de cada elemento mencionado, por ejemplo, las menciones de usuario pueden ser modificadas a un token especial, como por ejemplo `@user`, `@usuario`, `@usr`, o pueden ser eliminadas del texto. La transformación a tokens especiales permite que el modelo aprenda a identificar este tipo de estructuras.

In [5]:
preprocess_tweet(example_text, user_token='@usuario', url_token='url', preprocess_hashtags=True, demoji=False, shorten=2, normalize_laughter=True, hashtag_token='hashtag')

'Que opinas de la reformas Migratorias?👀👀 mi buen amigo @usuario?\nYo pienso que DEbieron facilitar la entrada de los inmigrantes 💁🇧🇷🇨🇷🇳🇮🇲🇽💪hashtag yo te apoyo eexx jaja. Página oficial del BID:  url'

Podemos observar que el texto cambia los links por la palabra definida "url", los nombres de usuario por "@usuario", separa las palabras contenidas en un hashtag y además agrega "hashtag", normaliza las expresiones de risa y las letras repetidas. Describe el contenido de cada emoticono.

Las palabras definidas serán muy útiles para el modelo, pues éste puede ser configurado para que considere dichas palabras como elementos estructurales y no como palabras.

Sin embargo, se observa que el texto contiene espacios, mantiene los acentos, los saltos de línea y las mayúsculas. Estas características podrían ser no deseadas según el objetivo que se tiene y el modelo que se empleará para clasificar los textos. Ya que existen modelos que hacen diferencia entre estas características y otros que no.

Para sustituir multiples espacios en blanco (incluye saltos de linea) por un solo espacio en blanco se usa la expresión regular `r'\s+'`. Para el siguiente ejemplo se sustituyen los espacios en blanco por el símbolo `#`.

In [6]:
#remove extra spaces
re.compile(r'\s+').sub('#', example_text)

'Que#opinas#de#la#reformas#Migratorias?👀👀#mi#buen#amigo#@Pepe?#Yo#pienso#que#DEbieron#facilitar#la#entrada#de#los#inmigrantes#💁🇧🇷🇨🇷🇳🇮🇲🇽💪#YoTeApoyo#eeeexxxxxx#jajajajaja.#Página#oficial#del#BID:#https://www.iadb.org/es#'

Juntemos estos conceptos para crear un método que nos permita limpiar el texto de cada tweet de manera automática.

In [7]:
def clean_tweet(tweet, user_token='@usuario', url_token='url', hashtag_token='hashtag', preprocess_hashtags=True, demoji=True, shorten=2, normalize_laughter=True):
    """Function to clean a tweet

    Args:
        tweet (str): text to clean
        user_token (str, optional): token to replace user mentions. Defaults to '@usuario'.
        url_token (str, optional): token to replace urls. Defaults to 'url'.
        hashtag_token (str, optional): token to replace hashtags. Defaults to 'hashtag'.
        preprocess_hashtags (bool, optional): if True, hashtags are preprocessed. Defaults to True.
        demoji (bool, optional): if True, emojis are replaced by their description. Defaults to True.
        shorten (int, optional): if > 0, words are shortened to the specified length. Defaults to 2.
        normalize_laughter (bool, optional): if True, laughter is normalized. Defaults to True.

    Returns:
        str: cleaned tweet
    """
    tweet = str(tweet)
    tweet = preprocess_tweet(tweet, user_token=user_token, url_token=url_token, preprocess_hashtags=preprocess_hashtags, demoji=demoji, shorten=shorten, normalize_laughter=normalize_laughter, hashtag_token=hashtag_token)
    tweet = re.compile(r'\s+').sub(' ', tweet)
    return tweet

#apply function to all tweets
data['text'] = data.text.apply(lambda x: clean_tweet(x))
#remove initial and final spaces
data['text'] = data.text.str.strip()

Comprobemos que el texto se ha limpiado correctamente, a través de la selección aleatoria de un texto.

In [8]:
data.sample(n=1).values

array([[1166364172050980864,
        '@usuario No. Considere la fuerza de trabajo en Chile total y el número de inmigrantes empleados . Ese ejercicio me dio algo como un 6.3. 12500 inmigrantes 6.6 % de la población 80% activos',
        '2021-11-22', 0]], dtype=object)

### División en conjuntos de entrenamiento, validación y prueba

La división en tres subconjuntos del conjunto de datos es de suma importancia, ya que cada uno de ellos juega un rol importante para la creación de un modelo de aprendizaje de máquinas.

* El conjunto de entrenamiento, es por lo regular, el que contiene más muestras, ya que a partir de estos datos el modelo debe aprender y/o generalizar las características lingüísticas de los tweets marcados como xenófobos y no xenófobos de la mejor forma posible. De esta manera, cuando el modelo deba clasificar un texto nuevo, su etiqueta sea correcta.

* El conjunto de validación, es por lo regular, el que contiene menos muestras. Este conjunto nos permitirá realizar múltiples evaluaciones sobre el modelo, ya sea para determinar si existen mejoras al momento de variar cualquier parámetro involucrado con el modelo, aumentar el conjunto de entrenamiento o incluso evaluar la selección de modelos.

* El conjunto de prueba nos permite realizar la evaluación final del modelo. Por lo regular este conjunto solo debe usarse una vez, esto para evitar posibles sesgos.

En el aprendizaje de máquinas es común aplicar el principio de Pareto, el cual sostiene que la división óptima es 80% de los datos para entrenamiento y el restante para tareas de evaluación. Sin embargo, dado que se sabe que el modelo pasará por una serie de algoritmos de optimización, sí se usara esta división podría lograrse un sesgo en cuanto al rendimiento real del modelo. 

Para la división en subconjuntos debe considerarse también la siguiente pregunta ¿El conjunto de datos total está balanceado? Al decir balanceado nos referimos a que las etiquetas disponibles son proporcionales entre sí. Para nuestro caso, dado que se trabaja con únicamente dos etiquetas, diremos que nuestro conjunto de datos es balanceado sí y sólo sí aproximadamente el 50% de los datos contienen la etiqueta *xenófobo*, y el restante corresponde a la etiqueta *no xenofóbo*.

Cuando el conjunto de datos se encuentra desbalanceado (como es el caso) la división en subconjuntos debe hacerse con cuidado, a modo de distribuir proporcionalmente la o las clases más desbalanceadas en cada uno de los tres.

Un ejemplo de omitir esta distribución puede suceder como sigue: Suponga que se tiene un conjunto de datos total de 1000 muestras, donde 780 corrresponden a la clase A y 220 a la clase B. Se hace la división de forma aleatoria a modo de que el conjunto de entrenamiento contiene sólo 20 muestras de la clase B y 680 de la clase A. Tanto el conjunto de validación y prueba contendrán 100 muestras de la clase A y 200 de la clase B. Dado que las muestras de la clase B en el conjunto de entrenamiento son pocas, el modelo no podrá aprender lo suficiente de la clase B y por ende, su rendimiento para esta clase será muy bajo.

In [9]:
#shuffle data before splitting for sanity
data = data.sample(frac=1, ignore_index=True)

#vamos a dividir el dataset por su id y etiqueta
X = data.id.values
y = data.label.values
#genera los conjuntos de entrenamiento y prueba, se reparte equitativamente las etiquetas en cada conjunto
x_train,x_test,y_train,y_test = train_test_split(X, 
                                    y, test_size=0.2, random_state=2022, stratify=y)
#de x_test y y_test, genera los conjuntos de prueba y validación
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.125, random_state=2022,
                                                  stratify=y_train)
                

Construye los conjuntos de entrenamiento, prueba y validación a partir de la división de los ids

In [10]:
train_df = data[data.id.isin(x_train)].reset_index(drop=True)
test_df = data[data.id.isin(x_test)].reset_index(drop=True)
valid_df = data[data.id.isin(x_val)].reset_index(drop=True)

A ojo veamos la distribución de las etiquetas en el conjunto de entrenamiento.

In [11]:
#check the distribution of the labels by eye
train_df.label.value_counts()

0    5469
1    1531
Name: label, dtype: int64

In [12]:
#check the final datasets
train_df.head(5)

,id,text,date,label
0,1256672684454359040,OJO Solidaridad emoji manos aplaudiendo emoji ...,2021-08-16,0
1,1086709855673629952,"@usuario @usuario Paga a hacienda, vuelve a Pe...",2021-07-22,0
2,1199252043774464000,@usuario Una muy buena noticia para nuestros h...,2021-11-15,0
3,1308390290458320896,@usuario Cresta! Pobre gente y pobre de nosotr...,2021-10-20,1
4,1347265781424398336,@usuario Si fueran migrantes estarían transmit...,2021-10-03,0


Guarda en un archivo de texto los conjuntos de entrenamiento, validación y prueba por separado, esto hará más sencillo el entrenamiento y evaluación del modelo.

In [13]:
#save the datasets
train_df.to_csv('./assets/data/train.csv', index=False)
test_df.to_csv('./assets/data/test.csv', index=False)
valid_df.to_csv('./assets/data/valid.csv', index=False)